In [2]:
import torch.nn as nn
import pandas as pd 
import torch.optim as optim
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import torch
from torch.utils.data import TensorDataset
from sklearn.model_selection import GridSearchCV


FILENAME = "train.csv"
df = pd.read_csv(FILENAME)

y = df.iloc[:, 0]   # Year
X = df.iloc[:, 1:]  # Features

# Definisci la rete neurale
class ImprovedNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ImprovedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Definisci la funzione di perdita e l'ottimizzatore
criterion = nn.MSELoss()

# Carica il dataset
# ... (Assicurati di avere X e y)

# Dividi il dataset in set di addestramento e di test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Converte i dati in tensori PyTorch
x_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

x_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Combina i dati in TensorDataset
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

# Definisci la griglia di ricerca degli iperparametri
param_grid = {
    'hidden_size': [16, 32, 64],
    'lr': [0.001, 0.01, 0.1]
}

# Imposta la validazione incrociata con KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Crea il modello PyTorch
model = ImprovedNN(input_size=X_train.shape[1], output_size=1)

# Creare un oggetto GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=kf, verbose=1)

# Addestra il modello con la griglia di ricerca
grid_search.fit(x_train_tensor, y_train_tensor)

# Ottieni il miglior modello
best_model = grid_search.best_estimator_

# Valuta il modello migliore sul set di test
best_model.eval()
y_pred = best_model(x_test_tensor)
mse = criterion(y_pred, y_test_tensor).item()

print(f'Mean Squared Error sul set di test usando il miglior modello: {mse}')


NameError: name 'TensorDataset' is not defined